# Quantum Order Finding

Import some useful Python packages

In [2]:
from qiskit import*
import math
import numpy as np

1. Given positive integers $b$ and $M$ such that $b < M$ and gcd($b,M$) = 1, write a classical algorithm to find the order $r$ of $b$ modulo $M$.

In [3]:
b = 7
M = 15
# Put your code here.
r=0
for i in range(1, M+1):
    if b ** i % M == 1:
        r= i
        break

# Output
print("The order of b=" + str(b) + " modulo M=" + str(M) + " is r=" + str(r))

The order of b=7 modulo M=15 is r=4


2. Given $b$ and $M$ as above and some integer $n$, print a table of $x$ and $y = b^x \; \mathsf{mod} \; M$ for $x \in \{0, \ldots, 2^n-1\}$.

In [4]:
n = 3
b = 7
M = 15
# Put your code here.
for i in range(1, 2**n):
    print('x is', i, 'y is', b** i % M)


x is 1 y is 7
x is 2 y is 4
x is 3 y is 13
x is 4 y is 1
x is 5 y is 7
x is 6 y is 4
x is 7 y is 13


For your convenience, below are the function definitions for the Quantum Fourier Transform (qft) and its inverse (iqft).

In [5]:
def qft(qc, q):
    # qc is a quantum circuit containing the quantum register q.
    # The function applies the Quantum Fourier Transform to q.
    
    # determine the number of qubits in the register
    n = len(q)
    
    # apply Hadamard gates and controlled rotations
    for j in range(n-1, -1, -1):
        qc.h(q[j])
        for k in range(j-1, -1, -1):
            qc.cu1(2*np.pi*2**(-(j-k+1)), q[k], q[j])
    
    # reverse the order of the qubits
    for l in range(int(np.floor(n/2))):
        qc.swap(q[l], q[n-1-l])

In [6]:
def iqft(qc, q):
    # qc is a quantum circuit containing the quantum register q.
    # The function applies the Inverse Quantum Fourier Transform to q.
    
    # determine the number of qubits in the register
    n = len(q)
    
    # reverse the order of the qubits
    for l in range(int(np.floor(n/2))):
        qc.swap(q[l], q[n-1-l])
    
    # apply Hadamard gates and controlled rotations
    for j in range(n):
        for k in range(j):
            qc.cu1(-2*np.pi*2**(-(j-k+1)), q[k], q[j])
        qc.h(q[j])

3. Define below the function $V_{7,15}$ per the circuit diagram on slide 7 of Lecture 17.  Note that $\oplus$ is a NOT gate.

In [17]:
def V7mod15(qc, qx, qy):
    # qc is a quantum circuit with quantum registers qx and qy
    # qx has at least 2 qubits; qy has at least 4 qubits
    # transforms |x, 1> to |x,y> = |x, 7**x % 15>
    qc.x(qy[2])
    
    qc.x(qx[0])
    qc.ccx(qx[0],qx[1],qy[0])
    qc.x(qx[0])
    
    qc.x(qx[1])
    qc.ccx(qx[0],qx[1],qy[1])
    qc.x(qx[1])
    
    qc.x(qx[0])
    qc.x(qx[1])
    qc.ccx(qx[0],qx[1],qy[2])
    qc.x(qx[0])
    qc.x(qx[1])
    
    qc.ccx(qx[0],qx[1],qy[3])
    
    
    
    
    
    
    
    
    
    # Put your code here.


The functions $V_{11,15}$ and $V_{4,21}$ are defined below for your convenience.

In [8]:
def V11mod15(qc, qx, qy):
    # qc is a quantum circuit with quantum registers qx and qy
    # qx has at least 1 qubit; qy has at least 4 qubits
    # transforms |x, 1> to |x,y> = |x, 11**x % 15>
    
    # x = 0 implies y = 0001
    # x = 1 implies y = 1011
    qc.cx(qx[0], qy[1])
    qc.cx(qx[0], qy[3])

In [9]:
def V4mod21(qc, qx, qy):
    # qc is a quantum circuit with quantum registers qx and qy
    # qx has at least 2 qubit; qy has at least 5 qubits
    # transforms |x, 1> to |x,y> = |x, 4**x % 21>
    
    qc.x(qy[0])
    # x = 00 implies y = 00001
    qc.x(qx[0])
    qc.x(qx[1])
    qc.ccx(qx[0], qx[1], qy[0])
    qc.x(qx[1])
    qc.x(qx[0])
    # x = 01 implies y = 00100
    qc.x(qx[1])
    qc.ccx(qx[0], qx[1], qy[2])
    qc.x(qx[1])
    # x = 10 implies y = 10000
    qc.x(qx[0])
    qc.ccx(qx[0], qx[1], qy[4])
    qc.x(qx[0])
    # x = 11 implies y = 00001
    qc.ccx(qx[0], qx[1], qy[0])

In [10]:
# Verify that your VbmodM function is working properly.

# choose n and m appropriately
n = 3
m = 5

# loop over all possible values of x
for x in range(2**n):
    
    # set up quantum registers qx, qy and classical registers cx, cy
    qx = qiskit.QuantumRegister(n)
    qy = qiskit.QuantumRegister(m)
    cx = qiskit.ClassicalRegister(len(qx))
    cy = qiskit.ClassicalRegister(len(qy))
    qc = qiskit.QuantumCircuit(qy, qx, cy, cx)
    
    # apply NOT gates to set the input register to x
    for k in range(n):
        if (math.floor(x/2**k) % 2 == 1):
            qc.x(qx[k])
    
    # initialize the output register to 1
    qc.x(qy[0])
    
    # apply the VbmodM function
    V7mod15(qc, qx, qy)
    
    # specify measurement gates for the input and output registers
    for i in range(len(qx)):
        qc.measure(qx[i], cx[i])
    for j in range(len(qy)):
        qc.measure(qy[j], cy[j])

    # execute the quantum circuit
    backend=qiskit.Aer.get_backend('qasm_simulator')
    job = qiskit.execute(qc, backend, shots=1024)
    data = job.result().get_counts(qc)
    print(data)
    
    # Compare the results to those of the table computed in part 2.

{'000 00101': 1024}
{'001 00101': 1024}
{'010 00101': 1024}
{'011 00101': 1024}
{'100 00011': 1024}
{'101 00011': 1024}
{'110 01100': 1024}
{'111 01100': 1024}


4. Implement the quantum order-finding algorithm and determine the value of r for each $V_{b,M}$ function.

In [19]:
# Put your code here.
n=2
m=4
qx = qiskit.QuantumRegister(n)
qy = qiskit.QuantumRegister(m)
c = qiskit.ClassicalRegister(n)
qc = qiskit.QuantumCircuit(qx,qy,c)
qc.x(qy[0])
qc.h(qx)
V7mod15(qc,qx,qy)
iqft(qc, qx)
for i in range(n):
    qc.measure(qx[i],c[i])
    
backend = Aer.get_backend('qasm_simulator')
job = execute(qc, backend, shots=1024)
result = job.result() 
counts=result.get_counts(qc)
print(counts)    
    

{'00': 279, '01': 245, '10': 246, '11': 254}


5. What value of $r$ do you get for each of the three functions $V_{7,15}$, $V_{11,15}$, and $V_{4,21}$?  Explain how you arrived at your answers.

For 7mod15, the LCM(4,2,4) turns out to be 4, which is a power of 2; 
the order from this algorithm is correct as 7^4mod15 is 1.

In [61]:
n=1
m=4
qx = qiskit.QuantumRegister(n)
qy = qiskit.QuantumRegister(m)
c = qiskit.ClassicalRegister(n)
qc = qiskit.QuantumCircuit(qx,qy,c)
qc.x(qy[0])
qc.h(qx)
V11mod15(qc,qx,qy)
iqft(qc, qx)
for i in range(n):
    qc.measure(qx[i],c[i])
    
backend = Aer.get_backend('qasm_simulator')
job = execute(qc, backend, shots=1024)
result = job.result() 
counts=result.get_counts(qc)
print(counts)    
      
    

{'1': 500, '0': 524}


The LCM(2) is 2. This works because it is a power of 2, and  11^2mod15 is 1.

In [59]:

n=2
m=5
qx = qiskit.QuantumRegister(n)
qy = qiskit.QuantumRegister(m)
c = qiskit.ClassicalRegister(n)
qc = qiskit.QuantumCircuit(qx,qy,c)
qc.x(qy[0])
qc.h(qx)
V4mod21(qc,qx,qy)
iqft(qc, qx)
for i in range(n):
    qc.measure(qx[i],c[i])
    
backend = Aer.get_backend('qasm_simulator')
job = execute(qc, backend, shots=1024)
result = job.result() 
counts=result.get_counts(qc)
print(counts)    

{'00': 408, '01': 243, '10': 129, '11': 244}


The algorithm fails in this case because r is not a power of 2. r is 3, and because s/r is equivalent to 1/2^p , there is no way 3 can be formed from 2^p. more calculations needed